In [1]:
import cv2
import os
import numpy as np
from pathlib import Path

In [2]:
base_dir     = Path("E:/face_recognition")
cascade_path = base_dir / "haarcascade_frontalface_default.xml"
data_dir     = base_dir / "known_faces"
threshold    = 100                    # Lower = stricter match
window_name  = "Live Face Recognition"


In [3]:
if not cascade_path.exists():
    raise FileNotFoundError(f"❌ Cascade XML not found: {cascade_path}")
face_cascade = cv2.CascadeClassifier(str(cascade_path))
if face_cascade.empty():
    raise RuntimeError("❌ Failed to load Haar cascade")

In [4]:
recognizer   = cv2.face.LBPHFaceRecognizer_create()
faces, labels, label_names = [], [], []
label_id = 0

if not data_dir.is_dir():
    raise FileNotFoundError(f"❌ known_faces folder not found: {data_dir}")

for person_folder in sorted(data_dir.iterdir()):
    if not person_folder.is_dir():
        continue
    label_names.append(person_folder.name)
    for img_file in person_folder.iterdir():
        img = cv2.imread(str(img_file))
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        for (x, y, w, h) in rects:
            faces.append(gray[y:y+h, x:x+w])
            labels.append(label_id)
    label_id += 1

if not faces:
    raise RuntimeError("❌ No faces found in known_faces/. Please add images.")

recognizer.train(faces, np.array(labels))
print(f"✅ Trained on {len(label_names)} people with {len(faces)} samples.")


✅ Trained on 3 people with 73 samples.


In [9]:

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise RuntimeError("❌ Cannot open webcam")

print(f"▶️ Streaming live recognition for project at {base_dir}")
print("   Press 'q' in the window to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Failed to grab frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    boxes = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in boxes:
        face_roi   = gray[y:y+h, x:x+w]
        label_id_p, conf = recognizer.predict(face_roi)
        if conf < threshold and 0 <= label_id_p < len(label_names):
            text = f"{label_names[label_id_p]} ({conf:.0f})"
        else:
            text = f"Unknown ({conf:.0f})"

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, text,
                    (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,
                    (0, 255, 0),
                    2)

    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

▶️ Streaming live recognition for project at E:\face_recognition
   Press 'q' in the window to quit.


In [10]:

# ─── 4. Cleanup & Exit ────────────────────────────────────────────────────────
cap.release()
cv2.destroyAllWindows()
print(f"✅ Closed all windows for project at {base_dir}")

✅ Closed all windows for project at E:\face_recognition
